In [1]:
from pyspark import SparkConf, SparkContext
import sys

In [2]:
conf = SparkConf().setMaster("local").setAppName("My App")
sc = SparkContext(conf = conf)
#sc.stop()

<H3>Q1. Consider the two data files (users.csv, transactions.csv).</H3>


Users file has the following fields: <br>
a) UserID <br>
b) EmailID <br>
c) NativeLanguage <br>
d) Location <br>

Transactions file has the following fields: <br>
a) Transaction_ID <br>
b) Product_ID <br>
c) UserID <br>
d) Price <br>
e) Product_Description <br>

*By making use of Spark Core (i.e. without using Spark SQL) find out:*

In [8]:
#Create rdd from csv file
users_data = sc.textFile('users.csv')
transactions_data = sc.textFile('transactions.csv')

users_rdd = users_data.map(lambda row : row.split(","))
transactions_rdd = transactions_data.map(lambda row : row.split(","))

users_df = users_data.toDF()

AttributeError: 'RDD' object has no attribute 'toDF'

In [6]:
type(users_rdd)

pyspark.rdd.PipelinedRDD

In [4]:
#create key value pair, with userId as key for both users and transactions data
users = users_rdd.map(lambda row : (int(row[0]), (row[1], row[2], row[3])))
transactions = transactions_rdd.map(lambda row : (int(row[2]), (row[0], int(row[1]), int(row[3]), row[4])))

In [5]:
#flattten the joined tables
def flatten_joined_list(row):
    user_id = row[0]
    return user_id, row[1][0][0], row[1][0][1], row[1][0][2], row[1][1][0], row[1][1][1], row[1][1][2], row[1][1][3]

In [6]:
#Join by userId and flatten the list
user_transactions = users.join(transactions)
user_transactions_rdd = user_transactions.map(lambda row : flatten_joined_list(row))

#check how joined rdd looks like
print(user_transactions_rdd.take(15))

[(2, 'user4@domain.com', 'EN', 'US', '12', 1004, 129, 'whatchamacallit'), (6, 'user17@website.net', 'FR', 'FR', '10', 1003, 89, 'gadget'), (10, 'user31@website.net', 'EN', 'CA', '2', 1001, 99, 'thingamajig'), (16, 'user53@school.edu', 'EN', 'US', '15', 1002, 149, 'gizmo'), (22, 'user71@domain.com', 'ES', 'MX', '9', 1001, 99, 'thingamajig'), (26, 'user85@service.io', 'HI', 'IN', '8', 1002, 149, 'gizmo'), (30, 'user99@website.net', 'EN', 'US', '7', 1002, 149, 'gizmo'), (1, 'user1@company.com', 'ES', 'MX', '11', 1004, 129, 'whatchamacallit'), (3, 'user5@company.com', 'FR', 'FR', '5', 1003, 89, 'gadget'), (5, 'user12@service.io', 'EN', 'CA', '13', 1005, 199, 'doohickey'), (7, 'user21@company.com', 'FR', 'FR', '14', 1004, 129, 'whatchamacallit'), (9, 'user27@school.edu', 'ES', 'MX', '4', 1001, 99, 'thingamajig'), (17, 'user57@school.edu', 'ES', 'MX', '3', 1004, 129, 'whatchamacallit'), (19, 'user64@school.edu', 'EN', 'US', '1', 1004, 129, 'whatchamacallit'), (19, 'user64@school.edu', 'EN', 

**a) Count of unique locations where each product is sold.**

In [7]:
#count by unique locations
count_by_location = user_transactions_rdd.map(lambda row : row[3]).countByValue()
for location, count in count_by_location.items() :
    print(location, count)

US 5
FR 3
CA 2
MX 4
IN 1


**b) Find out products bought by each user.**

In [8]:
product_by_user = user_transactions_rdd.map(lambda row : (row[0],row[5]))
product_by_user.collect()

[(2, 1004),
 (6, 1003),
 (10, 1001),
 (16, 1002),
 (22, 1001),
 (26, 1002),
 (30, 1002),
 (1, 1004),
 (3, 1003),
 (5, 1005),
 (7, 1004),
 (9, 1001),
 (17, 1004),
 (19, 1004),
 (19, 1002)]

**c) Total spending done by each user on each product.**

In [9]:
product_by_user = user_transactions_rdd.map(lambda row : (row[0], row[5], row[6]))
product_by_user.collect()

[(2, 1004, 129),
 (6, 1003, 89),
 (10, 1001, 99),
 (16, 1002, 149),
 (22, 1001, 99),
 (26, 1002, 149),
 (30, 1002, 149),
 (1, 1004, 129),
 (3, 1003, 89),
 (5, 1005, 199),
 (7, 1004, 129),
 (9, 1001, 99),
 (17, 1004, 129),
 (19, 1004, 129),
 (19, 1002, 149)]

<H3>Q2. Consider the dataset file Olympics.csv. </H3>

*This file contains information about the Olympic games, players participating in the games, and details of medals won by them. Using Spark core and the data file, compute the following:*

In [3]:
#Create RDD from file, separated by tabs
olympics = sc.textFile('olympics.csv').map(lambda row : row.split("\t"))
olympics.take(5)

[['Michael Phelps',
  '23',
  'United States',
  '2008',
  '08-24-08',
  'Swimming',
  '8',
  '0',
  '0',
  '8'],
 ['Michael Phelps',
  '19',
  'United States',
  '2004',
  '08-29-04',
  'Swimming',
  '6',
  '0',
  '2',
  '8'],
 ['Michael Phelps',
  '27',
  'United States',
  '2012',
  '08-12-12',
  'Swimming',
  '4',
  '2',
  '0',
  '6'],
 ['Natalie Coughlin',
  '25',
  'United States',
  '2008',
  '08-24-08',
  'Swimming',
  '1',
  '2',
  '3',
  '6'],
 ['Aleksey Nemov',
  '24',
  'Russia',
  '2000',
  '10-01-00',
  'Gymnastics',
  '2',
  '1',
  '3',
  '6']]

**a) Total medals that each country won in a particular sport (such as Gymnastics).**

In [11]:
medal_count_by_country = olympics.map(lambda row : ((row[2], row[5]), int(row[9])))
medal_count_by_country.reduceByKey(lambda key, count : key + count).sortByKey().collect()  
    
#medal_count_by_country.groupByKey().map(lambda row : row[0]).countByValue()

[(('Afghanistan', 'Taekwondo'), 2),
 (('Algeria', 'Athletics'), 5),
 (('Algeria', 'Boxing'), 1),
 (('Algeria', 'Judo'), 2),
 (('Argentina', 'Basketball'), 24),
 (('Argentina', 'Cycling'), 2),
 (('Argentina', 'Football'), 34),
 (('Argentina', 'Hockey'), 65),
 (('Argentina', 'Judo'), 1),
 (('Argentina', 'Sailing'), 10),
 (('Argentina', 'Swimming'), 1),
 (('Argentina', 'Taekwondo'), 1),
 (('Argentina', 'Tennis'), 3),
 (('Armenia', 'Boxing'), 1),
 (('Armenia', 'Weightlifting'), 5),
 (('Armenia', 'Wrestling'), 4),
 (('Australia', 'Archery'), 2),
 (('Australia', 'Athletics'), 16),
 (('Australia', 'Baseball'), 24),
 (('Australia', 'Basketball'), 48),
 (('Australia', 'Beach Volleyball'), 2),
 (('Australia', 'Canoeing'), 19),
 (('Australia', 'Cycling'), 36),
 (('Australia', 'Diving'), 17),
 (('Australia', 'Equestrian'), 10),
 (('Australia', 'Freestyle Skiing'), 5),
 (('Australia', 'Hockey'), 81),
 (('Australia', 'Judo'), 1),
 (('Australia', 'Rowing'), 61),
 (('Australia', 'Sailing'), 21),
 (('A

**b) In each Olympic games, how many medals has India won?**

In [12]:
#Filter out rows for India
olympics_india = olympics.filter(lambda x: 'India' in x)

#Take year and total medal count from filtered rdd and sum values using reduceByKey
count_by_year = olympics_india.map(lambda x : (x[3],int(x[9])))
count_by_year.reduceByKey(lambda year, count : year + count).collect()

[('2012', 6), ('2008', 3), ('2000', 1), ('2004', 1)]

**c) Compute top 3 countries in terms of total medals by each Olympic games year.**

In [40]:
olympics_by_year = olympics.map(lambda x : ((x[3], x[2]), int(x[9])))
#foo.map(lambda (x,y): (x, [y])).reduceByKey(lambda p,q: p+q).collect()
grouped_by_year = olympics_by_year.reduceByKey(lambda key, count : key + count).sortBy(lambda x: (x[0][0],x[1]), ascending=False)
#grouped_by_year

grouped_by_year.map(lambda x : (x[0][0],(x[0][1],x[1]))).foreach(print)

<H3>3. Consider the Movie Recommendation code - Please provide a brief write-up on the problem, steps needed to arrive at the solution (recommendation system), and how exactly those steps are implemented in the code.</H3>

In [14]:
import sys
from pyspark import SparkConf, SparkContext
from math import sqrt

Read pipe separated movie names and details. However only store movie id and name as key value pair in movieNames dictionary 

In [15]:
def loadMovieNames():
    movieNames = {}
    with open("../movies.txt") as f:
        for line in f:
            fields = line.split('|')
            print(fields)
            movieNames[int(fields[0])] = fields[1].encode().decode()#decode('ascii', 'ignore')
    return movieNames

create user ratings pair, for each movie rated by user. makePairs function joins movies 1, 2, and 3 rated by user as- 
(movie1,rating1), (movie1,rating1)
(movie1,rating1), (movie2,rating2)
(movie1,rating1), (movie3,rating3)

In [ ]:
def makePairs((user, ratings)):
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))

Remove all duplicate user, movie pair ratings. During cartesian join movie, ratings pairs gets repeated. For example -
(user1, ((movie1,rating1),(movie1,rating1))
(user1, ((movie1,rating1),(movie2,rating2))
(user1, ((movie1,rating1),(movie3,rating3))
(user1, ((movie2,rating2),(movie1,rating1))
(user1, ((movie2,rating2),(movie2,rating2))

Post applying this function, after removing duplicates, above pairs will becomes -
(user1, ((movie1,rating1),(movie2,rating2))
(user1, ((movie1,rating1),(movie3,rating3))

In [ ]:
def filterDuplicates( (userID, ratings) ):
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return movie1 < movie2

Below function calculate the similarity score between two movies. Here we are using consine similarity -
- Lesser the angle between two movies, more will be the value of cos theta
- If two movies are forming positive correlation, they can be recommended to user. If they are forming negative correlation, they shouldn't be recommended at all
- user's preferences for each of the movies together and adding those up

https://www.coursera.org/lecture/networks-illustrated/cosine-similarity-g1d3C
Ref : https://www.machinelearningplus.com/nlp/cosine-similarity/

In [ ]:
def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [22]:
conf = SparkConf().setMaster("local[*]").setAppName("MovieSimilarities")
sc = SparkContext(conf = conf)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:433)
	at sun.nio.ch.Net.bind(Net.java:425)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:223)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:128)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:558)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1283)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:501)
	at io.netty.channel.Abstract

In [ ]:
print("\nLoading movie names...")
nameDict = loadMovieNames()

Read the ratings file, and create ratings rdd - a key value pair such that key - [userId, (MovieId, Rating)]

In [ ]:
data = sc.textFile("file:///home/cloudera/moviedata/datafile2.txt")
ratings = data.map(lambda l: l.split()).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))

In [ ]:
joinedRatings = ratings.join(ratings)

uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)

moviePairs = uniqueJoinedRatings.map(makePairs)

moviePairRatings = moviePairs.groupByKey()

moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()

if (len(sys.argv) > 1):

    scoreThreshold = 0.10
    coOccurenceThreshold = 2

    movieID = int(sys.argv[1])

    filteredResults = moviePairSimilarities.filter(lambda((pair,sim)): \
        (pair[0] == movieID or pair[1] == movieID) \
        and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold)

    results = filteredResults.map(lambda((pair,sim)): (sim, pair)).sortByKey(ascending = False).take(10)

    print "Top 10 similar movies for " + nameDict[movieID]
    for result in results:
        (sim, pair) = result
        similarMovieID = pair[0]
        if (similarMovieID == movieID):
            similarMovieID = pair[1]
        print nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1])


In [19]:
sc.stop()